IMPORTANDO LIBRERIAS

In [33]:
import json  
import pandas as pd
import numpy as np
import re
import ast
import warnings
warnings.filterwarnings('ignore')
import pyarrow as pa
import pyarrow.parquet as pq

LEYENDO EL ARCHIVO JSON Y CONOCIENDO EL DATAFRAME

In [3]:
games = pd.read_json('output_steam_games.json', lines=True)
games.tail(5)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,1.49,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,None,"Nikita ""Ghost_RUS""",None,NaN,None,NaN
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,4.24,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,None,Sacada,None,NaN,None,NaN
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,None,Laush Dmitriy Sergeevich,None,NaN,None,NaN
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,None,"xropi,stev3ns",None,NaN,None,NaN
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,NaN,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None,None,None,NaN,None,NaN


AHORA CARGO EL ARCHIVO JSON LINEA POR LINEA

In [4]:
rows = []
with open('output_steam_games.json') as json_file:
    for line in json_file:
        rows.append(json.loads(line))

SE EXPANDE LA COLUMNA GENRES EN FILAS SEPARADAS Y CONSERVO AQUELLAS COLUMNAS CON INFORMACION UTIL PARA EL DESARROLLO DEL PROYECTO

In [5]:
games = pd.json_normalize(rows, record_path=['genres'], meta=['publisher', 'app_name','title','release_date','price','id','developer','items_count','metascore'])
games.shape

(71554, 10)

In [6]:
games.head(3)

,0,publisher,app_name,title,release_date,price,id,developer,items_count,metascore
0,Action,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,NaN,NaN
1,Casual,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,NaN,NaN
2,Indie,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,NaN,NaN


In [7]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71554 entries, 0 to 71553
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   0             71554 non-null  object
 1   publisher     59115 non-null  object
 2   app_name      71552 non-null  object
 3   title         71552 non-null  object
 4   release_date  71501 non-null  object
 5   price         68355 non-null  object
 6   id            71552 non-null  object
 7   developer     71204 non-null  object
 8   items_count   0 non-null      object
 9   metascore     5470 non-null   object
dtypes: object(10)
memory usage: 5.5+ MB


In [8]:
games.rename(columns={0:'genres'}, inplace=True)

 MEJORANDO LA CONSISTENCIA Y PRESENTACION DE DATOS DE AQUELLAS COLUMNAS DE TEXTO

In [9]:
games['genres'] = games['genres'].str.capitalize()
games['publisher'] = games['publisher'].str.capitalize()
games['app_name'] = games['app_name'].str.capitalize()
games['title'] = games['title'].str.capitalize()
games['developer'] = games['developer'].str.capitalize()

BUSCANDO REGISTROS DUPLICADOS, DE EXISTIR SE ELIMINAN

In [10]:
games[games.duplicated(keep=False)].size

20

In [12]:
games = games.drop_duplicates()
games.reset_index(drop=True,inplace=True)

In [13]:
games[games.duplicated(keep=False)].size

0

BUSCANDO CANTIDAD DE NULOS, DE HABER NULOS SE REEMPLAZARA POR SIN DATO

In [15]:
games.isnull().sum()

genres              0
publisher       12439
app_name            2
title               2
release_date       53
price            3199
id                  2
developer         350
items_count     71553
metascore       66084
dtype: int64

REEMPLAZANDO POR SIN DATO LOS VALORES NULOS

In [16]:
games['publisher'] = games['publisher'].replace({pd.NA: 'Sin Dato'})
games['app_name'] = games['app_name'].replace({pd.NA: 'Sin Dato'})
games['title'] = games['title'].replace({pd.NA: 'Sin Dato'})
games['release_date'] = games['release_date'].replace({pd.NA: ''})
games['metascore'] = games['metascore'].replace({pd.NA: ''})
games['developer'] = games['developer'].replace({pd.NA: 'Sin Dato'})

SE DEBE TRANSFORMAR LA COLUMNA PRICE A TIPO DE DATO FLOAT Y DE LA COLUMNA RELEASE_DATE DEBE OBTENERSE EL AÑO, LA COLUMNA ID SE RENOMBRA COMO ITEM_ID

In [17]:
games['free_play'] = games['price'].str.extract(r'([A-Za-z\s]+)')
games['free_play'] = games['free_play'].replace(np.nan, 'Sin Dato')
games['metascore'] = games['metascore'].replace('NA', 0)
games['price'] = pd.to_numeric(games['price'], errors='coerce')
games['metascore'] = pd.to_numeric(games['metascore'], errors='coerce')
games['price'] = games['price'].replace(np.nan, 0)
games['metascore'] = games['metascore'].replace(np.nan, 0)
games['id'] = games['id'].replace(np.nan, 0)
games.rename(columns={'id': 'item_id'}, inplace=True)

In [18]:
games['release_date'] = pd.to_datetime(games['release_date'], format='%Y-%m-%d', errors='coerce')
games['release_year'] = games['release_date'].dt.year
games['release_year'] = games['release_year'].astype('Int64')
games['release_year'].fillna(0, inplace=True)
games.drop(columns=['release_date'], inplace=True)

In [47]:
games.tail(3)

,genres,publisher,app_name,title,price,item_id,developer,items_count,metascore,free_play,release_year
70700,Simulation,Laush studio,Russian roads,Russian roads,1.99,610660,Laush dmitriy sergeevich,NaN,0.0,Sin Dato,2018
70701,Casual,Sixnails,Exit 2 - directions,Exit 2 - directions,4.99,658870,"Xropi,stev3ns",NaN,0.0,Sin Dato,2017
70702,Indie,Sixnails,Exit 2 - directions,Exit 2 - directions,4.99,658870,"Xropi,stev3ns",NaN,0.0,Sin Dato,2017


ELIMINO LOS REGISTROS DONDE RELEASE_YEAR ES CERO

In [22]:
games = games.drop(games[games['release_year']==0].index)
games = games.reset_index(drop=True)

In [24]:
games.isnull().sum()

genres              0
publisher           0
app_name            0
title               0
price               0
item_id             0
developer           0
items_count     70703
metascore           0
free_play           0
release_year        0
dtype: int64

In [30]:
games.isna().sum()

genres              0
publisher           0
app_name            0
title               0
price               0
item_id             0
developer           0
items_count     70703
metascore           0
free_play           0
release_year        0
dtype: int64

In [27]:
games.duplicated().sum()

0

In [45]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70703 entries, 0 to 70702
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        70703 non-null  object 
 1   publisher     70703 non-null  object 
 2   app_name      70703 non-null  object 
 3   title         70703 non-null  object 
 4   price         70703 non-null  float64
 5   item_id       70703 non-null  int64  
 6   developer     70703 non-null  object 
 7   items_count   0 non-null      object 
 8   metascore     70703 non-null  float64
 9   free_play     70703 non-null  object 
 10  release_year  70703 non-null  Int64  
dtypes: Int64(1), float64(2), int64(1), object(7)
memory usage: 6.0+ MB


In [44]:
games.to_parquet('ETLgames.parquet', engine= 'pyarrow')